# Author and Institution Networks 
This will be a data exploration of the networks which arrise when we explore the realtionships between and around authors and institiues in health infomatics

In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from pyvis.network import Network 
from IPython.display import display

In [ ]:
# import excel document into project
df = pd.read_csv(r"C:\Users\University\Downloads\review_data.csv",skipinitialspace = True)
df = pd.DataFrame(df)

In [ ]:
# List of columns to delete and then dropping them.
delete = ['Study ID','Covidence #','Reviewer Name','Title','Title.1','Type of publication','Study design','Architecture of the Digital Twin','Level of maturity','What is the purpose of the Digital twin', 'Application domain',  ]
df.drop(delete, axis=1, inplace=True)

In [ ]:
# move authors into their own rows
df['List of authors'] = df['List of authors'].str.split(';')
df = df.explode('List of authors')
# remove rows where 'List of authors' is null
df.dropna(subset=['List of authors'],inplace=True)
# remove rows where the length of the name is less than 1
df = df[df['List of authors'].str.len() >= 1]
display(df)

In [ ]:

# Step 1: Extract authors and co-authors
authors = df['List of authors'].tolist()
co_authors = df.groupby('Study ID.1')['List of authors'].apply(lambda x: x.tolist()).tolist()

# Step 2: Create a graph object
G = nx.Graph()

# Step 3: Add nodes
G.add_nodes_from(authors)

# Step 4: Add edges
for co_author_list in co_authors:
    for i in range(len(co_author_list)):
        for j in range(i+1, len(co_author_list)):
            author1 = co_author_list[i]
            author2 = co_author_list[j]
            if G.has_edge(author1, author2):
                # Increment edge weight if authors collaborated on multiple studies
                G[author1][author2]['weight'] += 1
            else:
                G.add_edge(author1, author2, weight=1)

# Step 5: Customize visualization
country_labels = df['Country in which the study conducted'].unique()
country_to_label = {country: label for label, country in enumerate(country_labels)}
node_colors = [country_to_label[df.loc[df['List of authors'] == author]['Country in which the study conducted'].iloc[0]] for author in G.nodes()]

node_sizes = [df.loc[df['List of authors'] == author].shape[0] for author in G.nodes()]  # Use number of studies as node sizes

# Step 6: Position nodes
pos = nx.kamada_kawai_layout(G)

# Step 7: Render and display the graph
plt.figure(figsize=(10, 10))
nx.draw_networkx(G, pos, node_color=node_colors, node_size=node_sizes, cmap='viridis')
plt.title('Authors Network')
plt.axis('off')
plt.show()
